In [25]:
import json
from pprint import pprint
import tinydb as tdb


## Requete OverPass turbo:

```sql
[out:json][timeout:500];
{{geocodeArea:France}}->.boundaryarea;

(
 node(area.boundaryarea)["ref:mhs"];
 way(area.boundaryarea)["ref:mhs"];
 relation(area.boundaryarea)["ref:mhs"];
);

out tags;
```
-- 20Mo

Rq: j'ai essayé avec `out meta;`, le fichier fait ~200Mo, et avec `out body;` ~90Mo... c'est surement mieux de séparer par département...


In [33]:
# Charge le fichier JSON:
filename = 'data/osmTags_France.json' 

with open(filename) as f:
    dataOsm = json.load(f)
    
dataOsm = dataOsm['elements']
len(dataOsm)

33071

In [27]:
# Garde seulement les monuments avec le code commençant par PA...
dataOsm = [ el for el in dataOsm if el['tags']['ref:mhs'].startswith('PA') ]
len(dataOsm)

32246

In [34]:
# Garde que les tags
dataOsm = [ el['tags'] for el in dataOsm ]

In [35]:
dataOsm[0]

{'heritage': '3',
 'heritage:operator': 'mhs',
 'historic': 'memorial',
 'image': "http://phototheque.1000wallpapers.com/galleries/Nature/p/Pa+%E0+Pl/Parc+de+la+T%EAte+d'Or/Ile+du+souvenir/Monument+aux+morts/large/Monument+aux+morts+5.jpg",
 'mhs:inscription_date': '1982',
 'name': "Monument aux morts de l'île du Souvenir",
 'ref:mhs': 'PA00117982',
 'wikidata': 'Q17341711',
 'wikipedia': "fr:Monument aux morts de l'île du Souvenir"}

In [36]:
# Push to tinyDB
db = tdb.TinyDB('data/tinyDB.json')
db.purge_table('osm')
osm = db.table('osm')
ids = osm.insert_multiple(dataOsm)
len(ids)

33071

In [ ]:
''' Convertie la liste en dictionaire pour les données OSM:
    key: ref. Mérimée
    value: tags osm
'''
dataOsm = { el['tags']['ref:mhs']:el for el in  dataOsm }

## Requete Wikidata
```sql
SELECT DISTINCT ?item ?itemDescription ?merimee ?article  WHERE {
  ?item wdt:P380 ?merimee .
  
  OPTIONAL { ?article schema:about ?item .
  ?article schema:isPartOf <https://fr.wikipedia.org/> . }
   
  FILTER(STRSTARTS(?merimee, "PA")).
  
  SERVICE wikibase:label {
     bd:serviceParam wikibase:language "fr" .
 }
}
```
-- 7,6 Mo

In [22]:
# Charge le fichier JSON:
filename = 'data/wikidata_France.json' 

with open(filename) as f:
    dataWiki = json.load(f)
    
len(dataWiki)

44912

In [23]:
dataWiki[0]

{'article': 'https://fr.wikipedia.org/wiki/%C3%89glise_Saint-%C3%89tienne-du-Mont_de_Paris',
 'item': 'http://www.wikidata.org/entity/Q583251',
 'itemDescription': 'édifice religieux de Paris, en France',
 'merimee': 'PA00088414'}

In [24]:
# Push to tinyDB
db = tdb.TinyDB('data/tinyDB.json')
db.purge_table('wikidata')
wikidata = db.table('wikidata')
ids = wikidata.insert_multiple(dataWiki)
len(ids)

44912

## Monuments sans page Wikipedia:

In [93]:
withoutAwikiPage = [ d for d in dataWiki if 'article' not in d ]
len(withoutAwikiPage)

23206

In [94]:
withoutAwikiPage[0]

{'item': 'http://www.wikidata.org/entity/Q1440262',
 'itemDescription': 'église à Vaugines (Vaucluse)',
 'merimee': 'PA84000019'}

## Comparaison Wikidata/Mérimée vs OSM

In [95]:
print( len(dataWiki) )
print( len(dataWiki) - len(dataOsm) )

44912
13552


Il semble bien avoir tout les monuments de Mérimés dans Wikidata...  
et il manquerait 13552 monuments dans OSM ?

## Monuments sans page Wikipedia et absent de OSM

In [96]:
noWikiAndNoOsm = [ d for d in withoutAwikiPage if d['merimee'] not in dataOsm ]
len(noWikiAndNoOsm)

8229

In [97]:
noWikiAndNoOsm[0]

{'item': 'http://www.wikidata.org/entity/Q1734360',
 'itemDescription': 'pont situé à Moet-sur-Loing (Seine-et-Marne)',
 'merimee': 'PA00087145'}

## Monuments AVEC une page Wikipedia mais absent de OSM

In [98]:
wikiButNoOsm = [ d for d in dataWiki if 
                    (d['merimee'] not in dataOsm) and ('article' in d) ]
len(wikiButNoOsm)

5229

In [99]:
wikiButNoOsm[0]

{'article': 'https://fr.wikipedia.org/wiki/Phare_de_Nividic',
 'item': 'http://www.wikidata.org/entity/Q406723',
 'itemDescription': 'phare français',
 'merimee': 'PA29000083'}